In [1]:
!pip install hvplot

In [14]:
# Import libraries
import talib as ta
from talib import abstract
from talib import MA_Type
import pandas as pd
import hvplot.pandas

In [3]:
# Read in data from csv
df = pd.read_csv('../Data/ETF_top_8_data.csv', header=[0, 1], parse_dates=True, index_col=[0])

In [83]:
# All availible function groups within TA-Lib
list(ta.get_function_groups())

['Cycle Indicators',
 'Math Operators',
 'Math Transform',
 'Momentum Indicators',
 'Overlap Studies',
 'Pattern Recognition',
 'Price Transform',
 'Statistic Functions',
 'Volatility Indicators',
 'Volume Indicators']

In [86]:
# Search for indicator by group
ta.get_function_groups()['Cycle Indicators']

['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE']

In [87]:
# Function allowing for customisable parameters in Bollinger Band
    # nbdevup = Deviation multiplier for upper band,
    # nbdevdn = Deviation multiplier for lower band,
    # Moving average type: simple moving average here
    # matype=0
def bbands_create_df(timeperiod, nbdevup, nbdevdn, ticker, matype):
    data=df[ticker]['close']
    bbands = ta.BBANDS(data, timeperiod, nbdevup, nbdevdn, matype)
    BB_df = pd.DataFrame(bbands).transpose()
    BB_df.rename(columns={0:'BB_UPPER', 1:'BB_MIDDLE', 2:'BB_LOWER'}, inplace=True)
    return BB_df

In [6]:
# Create a list of tickers that does not contain duplicates
ticker_list = list(dict.fromkeys((df.droplevel(axis=1, level=[-1]))))

In [68]:
# For loop that runs the Bollinger function for all tickers. It concatenates all Bollinger Band results into a single DataFrame, while also being indexed under their respective ticker
for x in ticker_list:
    # Create empty DataFrame
    empty_df = pd.DataFrame()
    # Run each Ticker through function
    bbands = bbands_create_df(timeperiod=5, nbdevup=2, nbdevdn=2, ticker=x, matype=0)
    # Create MultiIndex DataFrame containing the function results Indexed by the ticker name
    for BB_type in bbands.columns:    
        df[f'{x}', f'{BB_type}'] = bbands[BB_type]

# Re-sort index so that DataFrame is properly displayed
df = df.sort_index(axis=1)

In [69]:
# Preview DataFrame
df

GDX                                                        \
             BB_LOWER  BB_MIDDLE   BB_UPPER      close dividends       high   
Date                                                                          
2017-07-06        NaN        NaN        NaN  20.587141       0.0  20.788319   
2017-07-07        NaN        NaN        NaN  20.318905       0.0  20.520083   
2017-07-10        NaN        NaN        NaN  20.606304       0.0  20.740422   
2017-07-11        NaN        NaN        NaN  20.730841       0.0  20.826640   
2017-07-12  20.271354  20.613966  20.956578  20.826639       0.0  21.133195   
...               ...        ...        ...        ...       ...        ...   
2022-06-28  28.478923  29.398000  30.317077  28.870001       0.0  29.780001   
2022-06-29  28.188820  29.070000  29.951180  28.440001       0.0  29.270000   
2022-06-30  27.131108  28.764000  30.396892  27.379999       0.0  28.400000   
2022-07-01  27.065452  28.464000  29.862548  28.160000       0.0  28.370001   
2022-07-05  26.566325  27.962000  29.357675  26.959999       0.0  27.930000   

                                                         ...         XLV  \
                  low       open stock splits    volume  ...    BB_LOWER   
Date                                                     ...               
2017-07-06  20.539242  20.769159            0  37409500  ...         NaN   
2017-07-07  20.117728  20.453024            0  57304600  ...         NaN   
2017-07-10  20.108150  20.146471            0  52988200  ...         NaN   
2017-07-11  20.366806  20.615883            0  35247100  ...         NaN   
2017-07-12  20.788321  21.018237            0  39815200  ...   72.222279   
...               ...        ...          ...       ...  ...         ...   
2022-06-28  28.820000  29.719999            0  18484000  ...  123.738037   
2022-06-29  28.180000  29.160000            0  17375400  ...  126.389904   
2022-06-30  27.330000  28.230000            0  24080100  ...  127.051359   
2022-07-01  26.830000  26.959999            0  30974500  ...  126.966628   
2022-07-05  26.459999  27.680000            0  28993000  ...  127.064891   

                                                                      \
             BB_MIDDLE    BB_UPPER       close dividends        high   
Date                                                                   
2017-07-06         NaN         NaN   72.381187       0.0   73.152571   
2017-07-07         NaN         NaN   72.748520       0.0   72.831164   
2017-07-10         NaN         NaN   72.610756       0.0   72.803600   
2017-07-11         NaN         NaN   72.528130       0.0   72.720974   
2017-07-12   72.658522   73.094765   73.024017       0.0   73.262780   
...                ...         ...         ...       ...         ...   
2022-06-28  127.537997  131.337958  127.419998       0.0  130.589996   
2022-06-29  128.383997  130.378090  128.529999       0.0  129.169998   
2022-06-30  128.613998  130.176638  128.240005       0.0  128.970001   
2022-07-01  128.709998  130.453367  129.679993       0.0  129.850006   
2022-07-05  128.559998  130.055104  128.929993       0.0  128.960007   

                                                           
                   low        open stock splits    volume  
Date                                                       
2017-07-06   72.280173   73.014824            0   6367700  
2017-07-07   72.454661   72.638320            0   4371400  
2017-07-10   72.427097   72.776052            0   3580000  
2017-07-11   72.160804   72.564856            0   4117500  
2017-07-12   72.867906   72.886276            0   8808200  
...                ...         ...          ...       ...  
2022-06-28  127.180000  129.940002            0   7518700  
2022-06-29  127.389999  127.589996            0   9157700  
2022-06-30  126.889999  127.860001            0  10697600  
2022-07-01  126.860001  128.289993            0   9366600  
2022-07-05  126.169998  128.429993            0   9984900

In [88]:
# Prepare graphs for bollinger band
bb_upper = df['GDXJ']['BB_UPPER'].iloc[50:].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_middle = df['GDXJ']['BB_MIDDLE'].iloc[50:].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = df['GDXJ']['BB_LOWER'].iloc[50:].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

close = df['GDXJ']["close"].iloc[50:].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

In [90]:
# Overlay and plot graphs
bbands_plot = close * bb_upper * bb_middle * bb_lower
bbands_plot

:Overlay
   .Curve.Close     :Curve   [Date]   (close)
   .Curve.BB_UPPER  :Curve   [Date]   (BB_UPPER)
   .Curve.BB_MIDDLE :Curve   [Date]   (BB_MIDDLE)
   .Curve.BB_LOWER  :Curve   [Date]   (BB_LOWER)